In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import os
import datetime

import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

import json
import requests
import api

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

D:\ANACONDA3\envs\py38-env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [35]:
class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"

    def search_address(self, query, analyze_type=None, page=None, size=None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}

        if analyze_type != None:
            params["analyze_type"] = f"{analyze_type}"

        if page != None:
            params['page'] = f"{page}"

        if size != None:
            params['size'] = f"{size}"

        res = requests.get(self.URL_01, headers=self.headers, params=params)
        document = json.loads(res.text)

        return document
    
    def geo_coord2regioncode(self, x, y, input_coord=None, output_coord=None):
        """
        02 좌표-행정구역정보 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        if output_coord != None:
            params['output_coord'] = f"{output_coord}"
            
        res = requests.get(self.URL_02, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def geo_coord2address(self, x, y, input_coord=None):
        """
        03 좌표-주소 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
            
        res = requests.get(self.URL_03, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def geo_transcoord(self, x, y, output_coord, input_coord=None):
        """
        04 좌표계 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}",
                  "output_coord": f"{output_coord}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        res = requests.get(self.URL_04, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def search_keyword(self,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
        """
        05 키워드 검색
        """
        params = {"query": f"{query}"}
        
        if category_group_code != None:
            params['category_group_code'] = f"{category_group_code}"
        if x != None:
            params['x'] = f"{x}"
        if y != None:
            params['y'] = f"{y}"
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{params}"
        if sort != None:
            params['sort'] = f"{sort}"
        
        res = requests.get(self.URL_05, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
            
        
    def search_category(self, category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
        """
        06 카테고리 검색
        """
        params = {'category_group_code': f"{category_group_code}",
                  'x': f"{x}",
                  'y': f"{y}"}
        
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{size}"
        if sort != None:
            params['sort'] = f"{sort}"
            
        res = requests.get(self.URL_06, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document

In [36]:
# api key 사용자모듈 활용 불러오기
rest_api_key = api.kakao_api

kakao = KakaoLocalAPI(rest_api_key)

In [44]:
df = pd.read_csv('./원본/주정차시스템추출_지오코딩.csv', encoding='cp949')

In [47]:
df['h'] = None
for n in tqdm(range(len(df))):
    ti = df['time'][n].split(':')
    df['h'][n] = ti[0]

  0%|          | 0/1155 [00:00<?, ?it/s]

In [50]:
df = df.dropna(axis=0)

In [52]:
df.reset_index(drop=True, inplace=True)

In [55]:
hour_list = list(df['h'].unique())

In [56]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

df['geometry'] = df.apply(lambda row : Point([row['X'], row['Y']]), axis=1)

df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)

In [57]:
df

data      time           X          Y   h  \
0    2022-01-01   1:31:42  128.738827  35.300842   1   
1    2022-01-01   1:41:07  128.739322  35.300785   1   
2    2022-01-01   1:54:40  128.740164  35.301483   1   
3    2022-01-01   1:57:26  128.739574  35.300765   1   
4    2022-01-01   2:06:13  128.740164  35.301483   2   
..          ...       ...         ...        ...  ..   
935  2022-01-17  20:36:02  128.837970  35.234492  20   
936  2022-01-17  20:37:23  128.837970  35.234492  20   
937  2022-01-17  20:37:24  128.861905  35.238253  20   
938  2022-01-17  20:47:00  128.739586  35.309247  20   
939  2022-01-17  21:46:46  128.886227  35.239997  21   

                       geometry  
0    POINT (128.73883 35.30084)  
1    POINT (128.73932 35.30078)  
2    POINT (128.74016 35.30148)  
3    POINT (128.73957 35.30077)  
4    POINT (128.74016 35.30148)  
..                          ...  
935  POINT (128.83797 35.23449)  
936  POINT (128.83797 35.23449)  
937  POINT (128.86191 35.23825)  
938  POINT (128.73959 35.30925)  
939  POINT (128.88623 35.24000)  

[940 rows x 6 columns]

In [60]:
for n in tqdm(hour_list):
    df_ser = df[df['h']==n]
    
    # 폴더 생성
    try:
        directory = f'./전처리/point2/{n}시_MINW/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
    df_ser.to_file(f'{directory}{n}시_MINW.shp', encoding='cp949', driver='ESRI Shapefile')

  0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
df = pd.read_excel('./원본/주정차 CCTV 단속자료 (고정식).xlsx', sheet_name=2)

In [15]:
df['hour'] = None
for n in tqdm(range(len(df))):
    try:
        a = df['reg_date'][n]
        df['hour'][n] = a.hour
    except:
        continue

  0%|          | 0/32452 [00:00<?, ?it/s]

In [20]:
hour_list = list(df['hour'].unique())
hour_list

[10, 14, 11, 15, 7, 9, 13, 16, 18, 19, 20, 8, 21, 17, 12]

In [23]:
hour_list = list(df['hour'].unique())
hour_list

epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

df['geometry'] = df.apply(lambda row : Point([row['gps2'], row['gps1']]), axis=1)

df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)

In [24]:
df

uid    car_num       gps1        gps2       type  grade  \
0      10697521    48주9963  35.244808  128.875325  movcarchk  START   
1      10697522    48주9963  35.244743  128.875193  movcarchk  START   
2      10697525    22가8161  35.244550  128.874765  movcarchk  START   
3      10697524    96우1161  35.244640  128.874958  movcarchk  START   
4      10697528    71우8819  35.244517  128.874593  movcarchk  START   
...         ...        ...        ...         ...        ...    ...   
32447  10766303    82더2281  35.207655  128.852612  movcarchk  START   
32448  10766304  경남18바2434  35.188986  128.801353  movcarchk  START   
32449  10766305    23너2343  35.207738  128.852277  movcarchk  START   
32450  10766308    17두3037  35.190047  128.801086  movcarchk  START   
32451  10766309    70더5187  35.190090  128.801234  movcarchk  START   

                 reg_date Unnamed: 7          reg_date.1  Unnamed: 9  \
0     2022-01-01 10:52:00          토 2022-01-01 10:52:00          10   
1     2022-01-01 10:52:00          토 2022-01-01 10:52:00          10   
2     2022-01-01 10:52:00          토 2022-01-01 10:52:00          10   
3     2022-01-01 10:52:00          토 2022-01-01 10:52:00          10   
4     2022-01-01 10:52:00          토 2022-01-01 10:52:00          10   
...                   ...        ...                 ...         ...   
32447 2022-01-17 14:10:00          월 2022-01-17 14:10:00          14   
32448 2022-01-17 14:10:00          월 2022-01-17 14:10:00          14   
32449 2022-01-17 14:10:00          월 2022-01-17 14:10:00          14   
32450 2022-01-17 14:11:00          월 2022-01-17 14:11:00          14   
32451 2022-01-17 14:11:00          월 2022-01-17 14:11:00          14   

       Unnamed: 10 Unnamed: 11  Unnamed: 12  Unnamed: 13  Unnamed: 14  \
0              NaN           월       6599.0     0.203346          NaN   
1              NaN           화       5850.0     0.180266          NaN   
2              NaN           수       5884.0     0.181314          NaN   
3              NaN           목       5671.0     0.174750          NaN   
4              NaN           금       6230.0     0.191976          NaN   
...            ...         ...          ...          ...          ...   
32447          NaN         NaN          NaN          NaN          NaN   
32448          NaN         NaN          NaN          NaN          NaN   
32449          NaN         NaN          NaN          NaN          NaN   
32450          NaN         NaN          NaN          NaN          NaN   
32451          NaN         NaN          NaN          NaN          NaN   

      Unnamed: 15  Unnamed: 16  Unnamed: 17 hour                    geometry  
0              1시          0.0          0.0   10  POINT (128.87533 35.24481)  
1              2시          0.0          0.0   10  POINT (128.87519 35.24474)  
2              3시          0.0          0.0   10  POINT (128.87476 35.24455)  
3              4시          0.0          0.0   10  POINT (128.87496 35.24464)  
4              5시          0.0          0.0   10  POINT (128.87459 35.24452)  
...           ...          ...          ...  ...                         ...  
32447         NaN          NaN          NaN   14  POINT (128.85261 35.20766)  
32448         NaN          NaN          NaN   14  POINT (128.80135 35.18899)  
32449         NaN          NaN          NaN   14  POINT (128.85228 35.20774)  
32450         NaN          NaN          NaN   14  POINT (128.80109 35.19005)  
32451         NaN          NaN          NaN   14  POINT (128.80123 35.19009)  

[32452 rows x 20 columns]

In [32]:
df = df.astype({'reg_date':'str','reg_date.1':'str'})

In [34]:
for n in tqdm(hour_list):
    df_ser = df[df['hour']==n]
    
    # 폴더 생성
    try:
        directory = f'./전처리/point/{n}시_CCTV/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
    df_ser.to_file(f'{directory}{n}시_CCSV.shp', encoding='cp949', driver='ESRI Shapefile')

  0%|          | 0/15 [00:00<?, ?it/s]